In [5]:
import FinanceDataReader as fdr

name = "삼성전자"
df_krx = fdr.StockListing('KRX')
target = df_krx[df_krx['Name'] == name]
target

code = target.iloc[0]['Code']
price = target.iloc[0]['Close']
market_cap = target.iloc[0]['Marcap'] / 10**8  # 억 단위

print(f"종목명: {name}")
print(f"종목코드: {code}")
print(f"현재가: {price}원")
print(f"시가총액: {market_cap:.2f} 억원")


종목명: 삼성전자
종목코드: 005930
현재가: 152100원
시가총액: 9003769.28 억원


In [6]:
import FinanceDataReader as fdr
import requests
from bs4 import BeautifulSoup

def get_financial_data(code):
    """네이버 증권에서 주요 재무 지표를 크롤링합니다."""
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    data = {}
    try:
        # 네이버 증권의 주요재무정보 테이블(표)을 찾습니다.
        table = soup.select_one(".section.cop_analysis div.sub_section table")
        # 항목들 추출 (ROE, 부채비율, PSR 계산을 위한 매출액 등)
        # 실제 운영시에는 행/열 인덱스를 정교하게 매칭해야 합니다.
        rows = table.select("tbody tr")
        
        # 예시: ROE는 보통 6번째 행, 부채비율은 7번째 행에 위치 (연간 기준)
        data['roe'] = rows[5].select("td")[3].text.strip().replace(',', '')
        data['debt_ratio'] = rows[6].select("td")[3].text.strip().replace(',', '')
        data['sales'] = rows[0].select("td")[3].text.strip().replace(',', '') # 최근 매출액
    except Exception as e:
        print(f"데이터 추출 중 오류: {e}")
    return data

In [7]:
fin_data = get_financial_data("005930")
print(fin_data)

{'roe': '9.53', 'debt_ratio': '', 'sales': '3291027'}
